In [ ]:
import requests
import pandas as pd

# collecting DMO gilt announcements from 2005 to present

url = 'https://www.dmo.gov.uk/publications/?offset=0&itemsPerPage=1000000&parentFilter=1433&childFilter=1433|1450&startMonth=1&startYear=2005&endMonth=7&endYear=2021'
html = requests.get(url).content
df_list = pd.read_html(html)
dates = df_list[-1]
dates.to_excel('gilt_announcements_20052021.xlsx')

In [ ]:
# formatting dates dataset, some steps of data cleaning were done manually therefore I am importing the excel file

dates = dates.dropna()
dates = dates[dates['Publication title'].str.contains("Treasury Gilt")]
dates = dates[~dates['Publication title'].str.contains("Result")]
dates['amount'] = dates['Publication title'].str.split('Auction of ').str[1]
dates['amount2'] = dates['Publication title'].str.split('Auctions of ').str[1]
dates.loc[dates['amount'].isnull(), 'amount'] = dates['amount2']
dates['amount'] = dates['amount'].str.split('of').str[0]
dates.loc[dates['Publication title'].str.contains("Index"), 'index'] = 1
dates.loc[~dates['Publication title'].str.contains("Index"), 'index'] = 0
dates.amount.replace('\D+','',regex=True,inplace=True)
del dates['amount2']
dates = dates.dropna()
dates['Date published'] = pd.to_datetime(dates['Date published'],infer_datetime_format=True)

dates = pd.read_excel('datescheck4.xlsx')

In [ ]:
# formating covariates

exchange_rates = pd.read_csv('exchangerates.xlsm - ObservationData (1).csv')
exchange_rates = exchange_rates.T
new_header = exchange_rates.iloc[0] #grab the first row for the header
exchange_rates = exchange_rates[1:] #take the data less the header row
exchange_rates.columns = new_header
exchange_rates = exchange_rates.reset_index()
exchange_rates['index'] = pd.to_datetime(exchange_rates['index'] )
exchange_rates['Date'] = exchange_rates['index'].copy()
del exchange_rates['index']

ftse = pd.read_csv('ftse100close.csv').T
ftse = ftse[1:]
ftse = ftse.reset_index()
ftse.columns = ['Date','close']
ftse['close'] = ftse['close'].str.replace(',','.')
ftse = ftse.dropna()
ftse['close'] = pd.to_numeric(ftse['close'])
ftse['Date'] = pd.to_datetime(ftse['Date'],infer_datetime_format=True)

In [ ]:
# importing the breakeven rates datasets

spot_infla_2005 = pd.read_excel('Inflation_Daily_2005-2015.xlsx', '4')
spot_infla_2005 = spot_infla_2005.dropna()
spot_infla_pres = pd.read_excel('2016-present_infla.xlsx', '4')
spot_infla_pres = spot_infla_pres.dropna()

In [ ]:
# plotting 

implied_infla = implied_infla.set_index('Date')
# 5 year bonds
implied_infla[5].plot()

# 15 year bonds
implied_infla[15].plot()

# 25 year bonds 
implied_infla[25].plot()

In [ ]:
# merging with covariates

implied_infla = implied_infla.reset_index()
implied_infla = pd.merge(implied_infla, exchange_rates, on='Date')
implied_infla['Exchange Index'] = implied_infla['Exchange Index'].str.replace(r',', r'.').astype('float') 
implied_infla['Exchange Index'] = pd.to_numeric(implied_infla['Exchange Index'])


implied_infla = implied_infla.reset_index()
implied_infla = pd.merge(implied_infla, ftse, on='Date')
implied_infla['close'] = pd.to_numeric(implied_infla['close'])
implied_infla['close_pctchange'] = implied_infla['close'].pct_change()


In [ ]:
# more plotting
import numpy as np

implied_infla = implied_infla.set_index('Date')


np.log10(implied_infla[5]).plot()
np.log10(implied_infla['Exchange Index']).plot()
np.log10(implied_infla['close']).plot()

In [ ]:
# only using dates between the ones available with inflation expectations

dates = dates[dates['Date published'] <= '2021-03-31']
dates = dates[dates['Date published'] >= '2011-01-04']

In [ ]:
# getting the correlation matrix

a = implied_infla.corr()
a.to_excel('correlation_matrix.xlsx')

In [ ]:
# getting the pre-period and post-period dates

import warnings
warnings.filterwarnings('ignore')

from datetime import datetime
from dateutil.relativedelta import relativedelta

dates['pre_period'] = dates['Date published'] - pd.DateOffset(days=20)
dates['post_period'] = dates['Date published'] + pd.DateOffset(days=3)
dates['datepublished1'] = dates['Date published'] + pd.DateOffset(days=1)

from pandas.tseries.offsets import BDay
dates.pre_period = dates.pre_period.map(lambda x : x + 0*BDay())
dates.post_period = dates.post_period.map(lambda x : x + 0*BDay())
dates.datepublished1 = dates.datepublished1.map(lambda x : x + 0*BDay())
dates['Date published'] = dates['Date published'].map(lambda x : x + 0*BDay())

# pre-period and post-period dates should also be between bounds of inflation exp. range
dates = dates[dates['post_period'] <= '2021-03-31']
dates = dates[dates['pre_period'] >= '2011-01-04']
dates = dates.sort_values(by=['Date published'])

# transforming to lists for the main algorithm
implied_infla = implied_infla.reset_index()
a = dates['Date published'].tolist()
b = dates['pre_period'].tolist()
c = dates['post_period'].tolist()
d = dates['datepublished1'].tolist()

implied_infla['Date'] = pd.to_datetime(implied_infla['Date'],infer_datetime_format=True)
implied_infla = implied_infla[implied_infla['Date'] <= '2021-03-31']
implied_infla = implied_infla[implied_infla['Date'] >= '2011-01-04']
implied_infla = implied_infla.set_index("Date")

implied_infla.columns = [str(col) + '_year' for col in implied_infla.columns]
implied_infla.columns

In [ ]:
# creating the lists with all the years to be analyzed and their covariates 
year3 = implied_infla[['3_year','25_year','Exchange Index_year', 'close_year']]
year4 = implied_infla[['4_year','25_year','Exchange Index_year', 'close_year']]
year5 = implied_infla[['5_year','25_year','Exchange Index_year', 'close_year']]
year6 = implied_infla[['6_year','25_year','Exchange Index_year', 'close_year']]
year7 = implied_infla[['7_year','25_year','Exchange Index_year', 'close_year']]
year8 = implied_infla[['8_year','25_year','Exchange Index_year', 'close_year']]
year9 = implied_infla[['9_year','25_year','Exchange Index_year', 'close_year']]
year10 = implied_infla[['10_year','25_year','Exchange Index_year', 'close_year']]
year11 = implied_infla[['11_year','25_year','Exchange Index_year', 'close_year']]
year12 = implied_infla[['12_year','25_year','Exchange Index_year', 'close_year']]
year13 = implied_infla[['13_year','25_year','Exchange Index_year', 'close_year']]
year14 = implied_infla[['14_year','25_year','Exchange Index_year', 'close_year']]
year15 = implied_infla[['15_year','25_year','Exchange Index_year', 'close_year']]

list_years = [year3,year4,year5,year6,year7,year8,year9,year10,
             year11,year12,year13,year14,year15]

In [ ]:
# main BSTS algorithm

import numpy as np
import pandas as pd
from statsmodels.tsa.arima_process import ArmaProcess
from causalimpact import CausalImpact
import re
import warnings
warnings.filterwarnings('ignore')

res = dates[['Date published','pre_period', 'post_period','amount', 'index']]

for x in range(0, len(list_years)):
    print(list_years[x].columns[0])
    
    params = []
    errors = []
    list2 = []
    data = list_years[x].copy()
    data.columns = ['y', 'X1', 'X2', 'X3']
    
    for n in range(0,len(dates)):
        pre_period = [b[n], a[n]]
        post_period = [d[n], c[n]]
        try:
            ci = CausalImpact(data, pre_period, post_period)
            r1 = re.search('tail-area probability p: (.+?)\nPosterior prob.', ci.summary())
            params.append(r1.group(1))
            list2.append(ci.summary())

        except ValueError as err:
            params.append('NaN')
            errors.append(err)
            list2.append('error')
            pass
    
    try:
        print(len(params))
        print('sig numbers:', len([float(n) for n in params if float(n) <= 0.05]))
        print('non-sig numbers:', len([float(n) for n in params if float(n) > 0.05]))
        print(len(errors))
    except TypeError:
        print('weird')
    
    listtemp = []
    for num in range(0, len(list2)):
        try:
            actual_average = re.search('\nActual[\t ]*(.+?)[\t ]', list2[num]).group(1)
            actual_cumulative = re.search('\nActual[\t ]*\d.\d*[\t ]*(.+?)\nPrediction', list2[num]).group(1)

            rel_average = re.search('\n\nRelative effect \(s.d.\)[\t ]*(.+?)\s', list2[num]).group(1)
            rel_cumulative = re.search('\n\nRelative effect \(s.d.\).*-.*\)[\t ]*(.+?)\s', list2[num]).group(1)

            abs_average = re.search('\nAbsolute effect \(s.d.\)[\t ]*(.+?)\s', list2[num]).group(1)
            abs_cumulative = re.search('\nAbsolute effect \(s.d.\).*-.*\)[\t ]*(.+?)\s', list2[num]).group(1)

            listtemp.append([actual_average,actual_cumulative,rel_average,rel_cumulative,abs_average,abs_cumulative])
        
        except:
            listtemp.append(['error','error','error','error','error','error'])
            
    if x == 0:
        res['year3_p-value'] = params
        
        dtemp = pd.DataFrame.from_records(listtemp, columns =['actual_average','actual_cumulative','rel_average',
                                                              'rel_cumulative','abs_average','abs_cumulative'])
        dtemp.columns = [str(col) + 'year3' for col in dtemp.columns]
        
        res = pd.concat([res.reset_index(drop=True),dtemp.reset_index(drop=True)], axis=1)
        
    elif x == 1:
        res['year4_p-value'] = params
        dtemp = pd.DataFrame.from_records(listtemp, columns =['actual_average','actual_cumulative','rel_average',
                                                              'rel_cumulative','abs_average','abs_cumulative'])
        dtemp.columns = [str(col) + 'year4' for col in dtemp.columns]
        res = pd.concat([res.reset_index(drop=True),dtemp.reset_index(drop=True)], axis=1)
        
    elif x == 2:
        res['year5_p-value'] = params
        dtemp = pd.DataFrame.from_records(listtemp, columns =['actual_average','actual_cumulative','rel_average',
                                                              'rel_cumulative','abs_average','abs_cumulative'])
        dtemp.columns = [str(col) + 'year5' for col in dtemp.columns]
        res = pd.concat([res.reset_index(drop=True),dtemp.reset_index(drop=True)], axis=1)
    elif x == 3:
        res['year6_p-value'] = params
        dtemp = pd.DataFrame.from_records(listtemp, columns =['actual_average','actual_cumulative','rel_average',
                                                              'rel_cumulative','abs_average','abs_cumulative'])
        dtemp.columns = [str(col) + 'year6' for col in dtemp.columns]
        res = pd.concat([res.reset_index(drop=True),dtemp.reset_index(drop=True)], axis=1)
        
    elif x == 4:
        res['year7_p-value'] = params
        dtemp = pd.DataFrame.from_records(listtemp, columns =['actual_average','actual_cumulative','rel_average',
                                                              'rel_cumulative','abs_average','abs_cumulative'])
        dtemp.columns = [str(col) + 'year7' for col in dtemp.columns]
        res = pd.concat([res.reset_index(drop=True),dtemp.reset_index(drop=True)], axis=1)
    elif x == 5:
        res['year8_p-value'] = params
        dtemp = pd.DataFrame.from_records(listtemp, columns =['actual_average','actual_cumulative','rel_average',
                                                              'rel_cumulative','abs_average','abs_cumulative'])
        dtemp.columns = [str(col) + 'year8' for col in dtemp.columns]
        res = pd.concat([res.reset_index(drop=True),dtemp.reset_index(drop=True)], axis=1)
    elif x == 6:
        res['year9_p-value'] = params
        dtemp = pd.DataFrame.from_records(listtemp, columns =['actual_average','actual_cumulative','rel_average',
                                                              'rel_cumulative','abs_average','abs_cumulative'])
        dtemp.columns = [str(col) + 'year9' for col in dtemp.columns]
        res = pd.concat([res.reset_index(drop=True),dtemp.reset_index(drop=True)], axis=1)
    elif x == 7:
        res['year10_p-value'] = params
        dtemp = pd.DataFrame.from_records(listtemp, columns =['actual_average','actual_cumulative','rel_average',
                                                              'rel_cumulative','abs_average','abs_cumulative'])
        dtemp.columns = [str(col) + 'year10' for col in dtemp.columns]
        res = pd.concat([res.reset_index(drop=True),dtemp.reset_index(drop=True)], axis=1)
    
    elif x == 8:
        res['year11_p-value'] = params
        dtemp = pd.DataFrame.from_records(listtemp, columns =['actual_average','actual_cumulative','rel_average',
                                                              'rel_cumulative','abs_average','abs_cumulative'])
        dtemp.columns = [str(col) + 'year11' for col in dtemp.columns]
        res = pd.concat([res.reset_index(drop=True),dtemp.reset_index(drop=True)], axis=1)
        
    elif x == 9:
        res['year12_p-value'] = params
        dtemp = pd.DataFrame.from_records(listtemp, columns =['actual_average','actual_cumulative','rel_average',
                                                              'rel_cumulative','abs_average','abs_cumulative'])
        dtemp.columns = [str(col) + 'year12' for col in dtemp.columns]
        res = pd.concat([res.reset_index(drop=True),dtemp.reset_index(drop=True)], axis=1)
        
    elif x == 10:
        res['year13_p-value'] = params
        dtemp = pd.DataFrame.from_records(listtemp, columns =['actual_average','actual_cumulative','rel_average',
                                                              'rel_cumulative','abs_average','abs_cumulative'])
        dtemp.columns = [str(col) + 'year13' for col in dtemp.columns]
        res = pd.concat([res.reset_index(drop=True),dtemp.reset_index(drop=True)], axis=1)
    
    elif x == 11:
        res['year14_p-value'] = params
        dtemp = pd.DataFrame.from_records(listtemp, columns =['actual_average','actual_cumulative','rel_average',
                                                              'rel_cumulative','abs_average','abs_cumulative'])
        dtemp.columns = [str(col) + 'year14' for col in dtemp.columns]
        res = pd.concat([res.reset_index(drop=True),dtemp.reset_index(drop=True)], axis=1)   
        
    elif x == 12:
        res['year15_p-value'] = params
        dtemp = pd.DataFrame.from_records(listtemp, columns =['actual_average','actual_cumulative','rel_average',
                                                              'rel_cumulative','abs_average','abs_cumulative'])
        dtemp.columns = [str(col) + 'year15' for col in dtemp.columns]
        res = pd.concat([res.reset_index(drop=True),dtemp.reset_index(drop=True)], axis=1)        

In [ ]:
# T TESTS AND MORE RESULTS #

In [ ]:
# creating dataframes for all maturities

res3year = res[['Date published', 'pre_period', 'post_period','amount', 'index','year3_p-value', 
             'actual_averageyear3', 'actual_cumulativeyear3',
             'rel_averageyear3', 'rel_cumulativeyear3', 'abs_averageyear3','abs_cumulativeyear3']]
res4year = res[['Date published', 'pre_period', 'post_period','amount', 'index','year4_p-value', 
             'actual_averageyear4', 'actual_cumulativeyear4',
             'rel_averageyear4', 'rel_cumulativeyear4', 'abs_averageyear4','abs_cumulativeyear4']]
res5year = res[['Date published', 'pre_period', 'post_period','amount', 'index',
             'year5_p-value', 'actual_averageyear5',
       'actual_cumulativeyear5', 'rel_averageyear5', 'rel_cumulativeyear5',
       'abs_averageyear5', 'abs_cumulativeyear5']]
res6year = res[['Date published', 'pre_period', 'post_period','amount', 'index',
                'year6_p-value', 
             'actual_averageyear6', 'actual_cumulativeyear6',
             'rel_averageyear6', 'rel_cumulativeyear6', 'abs_averageyear6','abs_cumulativeyear6']]
res7year = res[['Date published', 'pre_period', 'post_period','amount', 'index',
             'year7_p-value','actual_averageyear7', 'actual_cumulativeyear7', 'rel_averageyear7',
       'rel_cumulativeyear7', 'abs_averageyear7', 'abs_cumulativeyear7']]
res8year = res[['Date published', 'pre_period', 'post_period','amount', 'index',
             'year8_p-value','actual_averageyear8', 'actual_cumulativeyear8', 'rel_averageyear8',
       'rel_cumulativeyear8', 'abs_averageyear8', 'abs_cumulativeyear8']]
res9year = res[['Date published', 'pre_period', 'post_period','amount', 'index',
             'year9_p-value','actual_averageyear9', 'actual_cumulativeyear9', 'rel_averageyear9',
       'rel_cumulativeyear9', 'abs_averageyear9', 'abs_cumulativeyear9']]
res10year = res[['Date published', 'pre_period', 'post_period','amount', 'index',
              'year10_p-value', 'actual_averageyear10', 'actual_cumulativeyear10',
       'rel_averageyear10', 'rel_cumulativeyear10', 'abs_averageyear10',
       'abs_cumulativeyear10']]
res11year = res[['Date published', 'pre_period', 'post_period','amount', 'index',
              'year11_p-value', 'actual_averageyear11', 'actual_cumulativeyear11',
       'rel_averageyear11', 'rel_cumulativeyear11', 'abs_averageyear11',
       'abs_cumulativeyear11']]
res12year = res[['Date published', 'pre_period', 'post_period','amount', 'index',
              'year12_p-value', 'actual_averageyear12', 'actual_cumulativeyear12',
       'rel_averageyear12', 'rel_cumulativeyear12', 'abs_averageyear12',
       'abs_cumulativeyear12']]
res13year = res[['Date published', 'pre_period', 'post_period','amount', 'index',
              'year13_p-value', 'actual_averageyear13', 'actual_cumulativeyear13',
       'rel_averageyear13', 'rel_cumulativeyear13', 'abs_averageyear13',
       'abs_cumulativeyear13']]
res14year = res[['Date published', 'pre_period', 'post_period','amount', 'index',
              'year14_p-value', 'actual_averageyear14', 'actual_cumulativeyear14',
       'rel_averageyear14', 'rel_cumulativeyear14', 'abs_averageyear14',
       'abs_cumulativeyear14']]
res15year = res[['Date published', 'pre_period', 'post_period','amount', 'index',
              'year15_p-value', 'actual_averageyear15', 'actual_cumulativeyear15',
       'rel_averageyear15', 'rel_cumulativeyear15', 'abs_averageyear15',
       'abs_cumulativeyear15']]

In [ ]:
# importing the data collected on the DMO pdfs

import pickle

with open("list1finally.txt", "rb") as fp:   # Unpickling
    data = pickle.load(fp)

#listdf = pd.DataFrame.from_records(data)

listdf = pd.read_excel('listdf3.xlsx')
listdf['Date published'] = pd.to_datetime(listdf['Date published'])
listdf.columns = ['Date published', 'avg_auction_value', 'supposed_auction_value_todate', 'total_sales_todate',
                             'total_auctions_planned','total_auctions_remaining','total_planned_sales','total_sales_remaining']

In [ ]:
# from here all the cells will be the same except for the different maturities

In [ ]:
res3year = res[['Date published', 'pre_period', 'post_period','amount', 'index','year3_p-value', 
             'actual_averageyear3', 'actual_cumulativeyear3',
             'rel_averageyear3', 'rel_cumulativeyear3', 'abs_averageyear3','abs_cumulativeyear3']]

res3year = res3year[~res3year['actual_averageyear3'].str.contains("error")]
res3year = res3year.dropna(subset = ['year3_p-value'])
res3year['rel_cumulativeyear3'] = res3year['rel_cumulativeyear3'].str.replace(r'%', r'').astype('float') / 100.0
res3year['rel_averageyear3'] = res3year['rel_averageyear3'].str.replace(r'%', r'').astype('float') / 100.0
res3year['actual_cumulativeyear3'] = pd.to_numeric(res3year['actual_cumulativeyear3'])
res3year['actual_averageyear3'] = pd.to_numeric(res3year['actual_averageyear3'])
res3year['abs_cumulativeyear3'] = pd.to_numeric(res3year['abs_cumulativeyear3'])
res3year['abs_averageyear3'] = pd.to_numeric(res3year['abs_averageyear3'])
res3year['year3_p-value'] = pd.to_numeric(res3year['year3_p-value'])
res3year['sig'] = 1
res3year.loc[res3year['year3_p-value'] > 0.05, 'sig'] = 0

df1 = res3year.merge(listdf, on='Date published')
df1['avg_auction_value'] = pd.to_numeric(df1['avg_auction_value'])
df1['amount'] = pd.to_numeric(df1['amount'])
df1['auctions_value_remaining'] = df1['total_sales_remaining']/(df1['total_auctions_remaining'])
df1['total_sales_to_date'] = df1['total_planned_sales'] - df1['total_sales_remaining']
df1['surprise_adaptive'] = df1['amount'] - df1['auctions_value_remaining']
df1['surprise_naive'] = df1['amount'] - df1['avg_auction_value']
df1['perc_surprise_adaptive'] = df1['amount']/df1['auctions_value_remaining']
df1['perc_surprise_naive'] = df1['amount']/df1['avg_auction_value']

df1_sig = df1[df1['sig'] == 1]
nominals = df1[df1['index'] == 0]

from scipy import stats
x = nominals[nominals['sig'] == 0]['surprise_adaptive']
y = nominals[nominals['sig'] == 1]['surprise_adaptive']
print('adaptive', stats.ttest_ind(x, y,equal_var=True))

x = nominals[nominals['sig'] == 0]['surprise_naive']
y = nominals[nominals['sig'] == 1]['surprise_naive']
print('naive', stats.ttest_ind(x, y,equal_var=True))
print(''' 
''')
print('mean of relative effect: ', 
      nominals[nominals['sig'] == 1]['rel_averageyear3'].mean())
print(''' 
''')
print('mean of surprise of non-significant gilt announcements: ', 
      nominals[nominals['sig'] == 0]['surprise_naive'].mean())
print('mean of surprise of significant gilt announcements: ', 
      nominals[nominals['sig'] == 1]['surprise_naive'].mean())
print(''' 
''')
print('number of significant:', len(y))
print('number of non-significant:', len(x))
print(''' 
''')
1/(nominals[nominals['sig'] == 1]['surprise_naive'].mean()/nominals[nominals['sig'] == 0]['surprise_naive'].mean())